<a href="https://colab.research.google.com/github/Lucassius/Daydreams/blob/main/ResolucaoDoProblemaTransporteDeAlunoseEscolas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabalho 1

O trabalho consistem em realizar o transporte de alunos e escolas no menor custo possível. Vocês podem definir o melhor comprimisso entre número de veículos e distância percorrida por veículo

Para isso é necessário baicar uma base de dados com os dados dos alunos, das escolas e as distãncias entre quaisquer dois pares de vértices

In [1]:
# download da base de dados
!wget https://github.com/lucasvra/domotica_robotica/raw/main/dados/trabalho.sqlite

--2023-05-07 22:32:45--  https://github.com/lucasvra/domotica_robotica/raw/main/dados/trabalho.sqlite
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/lucasvra/domotica_robotica/main/dados/trabalho.sqlite [following]
--2023-05-07 22:32:45--  https://raw.githubusercontent.com/lucasvra/domotica_robotica/main/dados/trabalho.sqlite
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16101376 (15M) [application/octet-stream]
Saving to: ‘trabalho.sqlite’

trabalho.sqlite     100%[===================>]  15.36M  --.-KB/s    in 0.1s    

2023-05-07 22:32:45 (133 MB/s) - ‘trabalho.sqlite’ saved [16101376/16101

O dados são organizados em três tabelas, alunos_req, escolas_req e distancias_req. Cada aluno tem um vértice_orig que é seu ponto de partida. Seu vértice de destino deve ser o vértice da escola ao qual ele pertence.

A distância é data tanto em metros quanto em minutos. Neste trabalho deve-se utilizar a distância em metros.

In [2]:
import pandas as pd
import numpy as np
import sqlite3

con = sqlite3.connect("trabalho.sqlite")
alunos = pd.read_sql_query("SELECT * from alunos_req", con)
escolas = pd.read_sql_query("SELECT * from escolas_req", con)
distancias = pd.read_sql_query("SELECT * from distancias_req", con)

In [3]:
alunos.head()

,cod_aluno,vertice_orig,dist_perc,turno,cod_escola,tipo,ensino
0,2496080,7222,5.252289,M,31173053,M,EF
1,2496081,3135,12.709945,M,31173053,M,PE
2,2496083,7247,26.376901,M,31173053,M,EF
3,2496084,7264,19.606927,M,31173053,M,EF
4,2496090,2750,14.089380,M,31173053,M,EF


In [4]:
escolas.head()

,vertice,entrada_m,saida_m,entrada_t,saida_t,entrada_n,saida_n,cod_escola
0,5339,07:30:00,11:30:00,12:30:00,16:45:00,18:00:00,22:15:00,311723
1,4604,07:00:00,11:10:00,13:00:00,17:00:00,19:00:00,23:00:00,31170674
2,7376,07:30:00,11:25:00,13:00:00,17:25:00,18:30:00,22:00:00,31173045
3,3997,07:20:00,11:35:00,13:00:00,17:15:00,18:00:00,22:15:00,31173053
4,7375,07:00:00,11:25:00,13:00:00,17:25:00,18:30:00,22:00:00,31173061


In [5]:
distancias.head()

,origem,destino,distancia,tempo
0,2763,300,11478.7640,22.910070
1,2763,2710,7106.7485,14.213497
2,2763,2750,2388.5647,4.777130
3,2763,2920,23322.0430,36.493070
4,2763,2970,24995.4750,36.925260


O problema deve ser resolvido utilizando algoritmos genéticos ou alguma variação deles (algoritmos evolucionários, VNS, clonal, etc)

Para isso se faz necessário codificar a solução num formato de genoma.

Considerando que temos n alunos, uma solução é codificada como uma matriz 2xn, ou seja com duas linhas e n colunas.

A primeira linha é composta do índice do aluno no dataframe alunos_escola e a segunda linha é o índice do veículo usado pra transporta esse aluno.

A solução inicial é considerar um veículo por aluno, então a solução seria algo no sentido.

[ 1, 2, 3, ... n-1]

[ 1, 2, 3, ... n-1]
 
Ou seja trocando as posições dos alunos na primeira linha, trocamos a ordem com que eles são levados à escola, trocando 

In [6]:
for index, row in escolas.iterrows():
  alunos_escola = alunos[alunos.cod_escola == row['cod_escola']]
  n = len(alunos_escola.index)
  print(f"escola {row['cod_escola']} | {n} alunos")

  sol = np.array([[*range(n)], [*range(n)]])

escola 311723 | 4 alunos
escola 31170674 | 5 alunos
escola 31173045 | 34 alunos
escola 31173053 | 23 alunos
escola 31173061 | 186 alunos
escola 31173088 | 2 alunos
escola 31173169 | 11 alunos
escola 31173185 | 16 alunos
escola 31246115 | 58 alunos
escola 31303330 | 63 alunos
escola 31306967 | 26 alunos
escola 31313467 | 41 alunos
escola 931246115 | 4 alunos


A função custo considera alguns fatores, em primeiro lugar, cada veículo custa R\$ 1.000,00 e cada km percorrido custa R\$ 2,00

Para isso é necessário encontrar os veículos, calcular a distância percorrida de cada um e somar tudo.

Uma regra importante é que não podem ser transportado mais do que 8 alunos por veículo. Qualquer solução que gere mais que 8 aluno por veículo é considerada infactível.

In [7]:
def distancia(origem, destino):
  if origem == destino: return 0.0
  d = distancias.query(f"origem == {origem} and destino == {destino}")
  return d.iloc[0].distancia/1000.0 if d.size > 0 else float('inf')

def fitness(sol, alunos_escola, escola):
  custo_total = 0

  #print(f"\ta solucao tem {np.unique(sol[1,:]).size} veiculos")

  for veiculo in np.unique(sol[1,:]):
    alunos_veiculo = sol[0,sol[1,:]==veiculo]

    if alunos_veiculo.size > 8:
      return float('inf') # não podem ter mais que 8 alunos por veículo

    custo_veiculo = 1000 # 1000 reais por veículo

    vertice = alunos_escola.iloc[alunos_veiculo[0].item()].vertice_orig
    dist = 0
    for al in alunos_veiculo[1:]:
      novo_vertice = alunos_escola.iloc[al].vertice_orig
      dist += distancia(vertice, novo_vertice)
      vertice = novo_vertice

    dist += distancia(vertice, escola.vertice)
    custo_veiculo += dist * 2 # 2 reais por km
    custo_total += custo_veiculo

  #print(f"\tcusto: {custo_total}")
  return custo_total

In [8]:
for index, escola in escolas.iterrows():
  alunos_escola = alunos[alunos.cod_escola == escola['cod_escola']]
  n = len(alunos_escola.index)
  print(f"escola {escola.cod_escola} | {n} alunos")

  sol = np.array([[*range(n)], [*range(n)]])
  fitness(sol, alunos_escola, escola)

escola 311723 | 4 alunos
escola 31170674 | 5 alunos
escola 31173045 | 34 alunos
escola 31173053 | 23 alunos
escola 31173061 | 186 alunos
escola 31173088 | 2 alunos
escola 31173169 | 11 alunos
escola 31173185 | 16 alunos
escola 31246115 | 58 alunos
escola 31303330 | 63 alunos
escola 31306967 | 26 alunos
escola 31313467 | 41 alunos
escola 931246115 | 4 alunos


A ideia do trabalho consiste em modificar essa solução ou família de soluções simplesmente diminuindo o número de veículos ou trocando alunos de posição

# Relatório
Como relatório é importante mostrar uma solução completa do problema, mostrando custo total de transporte para todas as escolas e todos os alunos, além de detalhar todas as viagens, incluindo a distância percorrida, o custo e a ordem de coleta dos alunos.

Além disso, é importante explicar a lógica do código/estratégia desenvolvido e fazer uma análise crítica dos resultados obtidos.

In [9]:
import random

def muda_veiculo(sol):
  novo = sol.copy()
  l, n = novo.shape

  pos = random.randint(0,n-1)
  veiculo = random.randint(0,n-1)

  #print(f"pos: {pos} | veic {veiculo}")

  novo[1,pos] = veiculo

  #print(novo)

  return novo

In [13]:

for index, escola in escolas.iterrows():
  alunos_escola = alunos[alunos.cod_escola == escola['cod_escola']]
  n = len(alunos_escola.index)
  print(f"escola {escola.cod_escola} | {n} alunos")

  sol = np.array([[*range(n)], [*range(n)]])
  custo = fitness(sol, alunos_escola, escola)
  max_sem_melhora = 100
  it_sem_melhora = 0

  for it in range(1000):
    nsol = muda_veiculo(sol)
    ncusto = fitness(nsol, alunos_escola, escola)

    if ncusto < custo:
      sol = nsol
      custo = ncusto
      it_sem_melhora = 0
    else:
      it_sem_melhora += 1

    if it_sem_melhora >= max_sem_melhora:
      break
  
  print(sol)
  print(f"\tordem de coleta:")
  for veiculo in np.unique(sol[1,:]):
    alunos_veiculo = sol[0,sol[1,:]==veiculo]
    vertice = alunos_escola.iloc[alunos_veiculo[0].item()].vertice_orig
    print(f"\t\tVeículo {veiculo}: escola -> Aluno {alunos_veiculo[0]}")
    for al in alunos_veiculo[1:]:
      novo_vertice = alunos_escola.iloc[al].vertice_orig
      dist = distancia(vertice, novo_vertice)
      print(f"\t\t          -> Aluno {al} ({dist:.1f} km)")
      vertice = novo_vertice
    print(f"\t\t          -> escola ({distancia(vertice, escola.vertice):.1f} km)")

  print(f'\tcusto da solucao: R${custo:.2f}\n')



escola 311723 | 4 alunos
[[0 1 2 3]
 [1 1 1 1]]
	ordem de coleta:
		Veículo 1: escola -> Aluno 0
		          -> Aluno 1 (0.0 km)
		          -> Aluno 2 (8.0 km)
		          -> Aluno 3 (2.5 km)
		          -> escola (0.6 km)
	custo da solucao: R$1022.23

escola 31170674 | 5 alunos
[[0 1 2 3 4]
 [3 3 3 3 3]]
	ordem de coleta:
		Veículo 3: escola -> Aluno 0
		          -> Aluno 1 (37.3 km)
		          -> Aluno 2 (46.7 km)
		          -> Aluno 3 (0.6 km)
		          -> Aluno 4 (41.3 km)
		          -> escola (19.7 km)
	custo da solucao: R$1291.28

escola 31173045 | 34 alunos
[[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
  24 25 26 27 28 29 30 31 32 33]
 [25 21 17 25 26  5  9  9  8  5 11 11  9 21 17 25  7 17  9 21 33 26  7 21
  21 25 26 17  7  8 11 21 11 33]]
	ordem de coleta:
		Veículo 5: escola -> Aluno 5
		          -> Aluno 9 (15.2 km)
		          -> escola (1.9 km)
		Veículo 7: escola -> Aluno 16
		          -> Aluno 22 (15.2 km)
		          -> Aluno 28 (17.